In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

albany au
vila velha br
nhulunbuy au
koumac nc
tuktoyaktuk ca
puerto ayora ec
tsihombe mg
araguatins br
kapaa us
dondo mz
port augusta au
hobart au
jamestown sh
east london za
albany au
mys shmidta ru
punta arenas cl
rikitea pf
jamestown sh
rikitea pf
hambantota lk
birjand ir
troitskoye ru
namatanai pg
mataura pf
barentsburg sj
chokurdakh ru
carnarvon au
arraial do cabo br
te anau nz
lagoa pt
kapaa us
atuona pf
mataura pf
longyearbyen sj
alotau pg
avarua ck
rikitea pf
fortuna us
qaanaaq gl
tuktoyaktuk ca
madimba tz
ilulissat gl
ushuaia ar
rikitea pf
qafsah tn
sambava mg
dawei mm
zhangjiakou cn
los llanos de aridane es
saldanha za
jamestown sh
lebu cl
new norfolk au
porosozero ru
vysokogornyy ru
kautokeino no
tacna pe
nikolskoye ru
kapaa us
carnarvon au
atuona pf
punta arenas cl
busselton au
margate za
busselton au
singapore sg
arraial do cabo br
port hawkesbury ca
dikson ru
mar del plata ar
paramonga pe
vaini to
nome us
rikitea pf
rikitea pf
ushuaia ar
fare pf
bluff nz
ribeira grande p

cape town za
ushuaia ar
butaritari ki
barentsburg sj
moron mn
tuktoyaktuk ca
singkang id
deputatskiy ru
halalo wf
bengkulu id
yellowknife ca
hobart au
rikitea pf
rikitea pf
lokosovo ru
chokurdakh ru
vaitupu wf
flinders au
sao filipe cv
new norfolk au
yuanli tw
bluff nz
taolanaro mg
kodiak us
ushuaia ar
tuktoyaktuk ca
hobart au
kapaa us
sorland no
touros br
ponta delgada pt
sungaipenuh id
tombouctou ml
taree au
emerald au
bredasdorp za
upernavik gl
kapit my
owando cg
punta arenas cl
upernavik gl
marivan ir
erdemli tr
maceio br
bambous virieux mu
ushuaia ar
constitucion mx
oranjemund na
vaini to
ushuaia ar
kaitangata nz
rikitea pf
tsihombe mg
ushuaia ar
hobart au
butaritari ki
srednekolymsk ru
grand river south east mu
rikitea pf
anito ph
port alfred za
flinders au
severo-kurilsk ru
tumannyy ru
punta arenas cl
east london za
atuona pf
plouzane fr
punta arenas cl
kodiak us
albany au
cape town za
singkang id
burnie au
albany au
saint-pierre re
bambous virieux mu
yellowknife ca
avarua ck
us